In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import glob

In [2]:
glob.glob('ml-latest-small/*')

['ml-latest-small/links.csv',
 'ml-latest-small/tags.csv',
 'ml-latest-small/ratings.csv',
 'ml-latest-small/README.txt',
 'ml-latest-small/movies.csv']

In [3]:
links = pd.read_csv("ml-latest-small/links.csv")
tags = pd.read_csv("ml-latest-small/tags.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

In [4]:
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [5]:
tags.head(2)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996


In [6]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [7]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [8]:
def split(x):
    y = x.split('|')
    return y

In [9]:
temp = pd.DataFrame()
movieIds = []
genres = []
for idx, i in enumerate(movies['genres'].apply(split)):
    for j in i:
        movieIds.append(movies.iloc[idx,0])
        genres.append(j)

In [10]:
temp['movieId'] = movieIds
temp['genres'] = genres

In [11]:
temp['genres'].value_counts()

Drama                 4361
Comedy                3756
Thriller              1894
Action                1828
Romance               1596
Adventure             1263
Crime                 1199
Sci-Fi                 980
Horror                 978
Fantasy                779
Children               664
Animation              611
Mystery                573
Documentary            440
War                    382
Musical                334
Western                167
IMAX                   158
Film-Noir               87
(no genres listed)      34
Name: genres, dtype: int64

In [12]:
temp2 = pd.merge(temp,ratings.drop(['userId', 'timestamp'], axis=1),how="inner", on="movieId")

In [13]:
finalData = pd.merge(temp2,tags.drop(['userId', 'timestamp'], axis=1),how="inner", on="movieId")

In [14]:
finalData.head()

,movieId,genres,rating,tag
0,1,Adventure,4.0,pixar
1,1,Adventure,4.0,pixar
2,1,Adventure,4.0,fun
3,1,Adventure,4.0,pixar
4,1,Adventure,4.0,pixar


In [15]:
finalData['rating'] = finalData['rating'].astype(object)

In [16]:
from sklearn.preprocessing import LabelEncoder
leGeneres = LabelEncoder()
leRating = LabelEncoder()
leTags = LabelEncoder()

In [17]:
leGeneres.fit(finalData.genres)

LabelEncoder()

In [18]:
leRating.fit(finalData.rating)

LabelEncoder()

In [19]:
leTags.fit(finalData.tag)

LabelEncoder()

In [20]:
finalDataEncoded = pd.DataFrame()

In [21]:
finalDataEncoded['genres'] = leGeneres.transform(finalData.genres)

In [22]:
finalDataEncoded['rating'] = leRating.transform(finalData.rating)

In [23]:
finalDataEncoded['tag'] = leTags.transform(finalData.tag)

In [24]:
finalDataEncoded

,genres,rating,tag
0,2,7,1239
1,2,7,1239
2,2,7,930
3,2,7,1239
4,2,7,1239
...,...,...,...
765266,5,6,1316
765267,16,6,604
765268,16,6,766
765269,16,6,946


In [25]:
X = finalDataEncoded.drop(['genres'], axis=1)
y = finalDataEncoded[['genres']]

In [26]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()

In [ ]:
logReg.fit(X,y)

In [ ]:
movieTag = ["thought-provoking"]
movieRating = [4.0]

In [ ]:
movieTagTransformed = leTags.transform(movieTag)
movieRatingTransformed = leRating.transform(pd.Series(movieRating, dtype=object))

In [ ]:
movieTagTransformed, movieRatingTransformed

In [ ]:
pred = pd.DataFrame()
pred['rating'] = movieRatingTransformed
pred['tag'] = movieTagTransformed

In [ ]:
pred

In [ ]:
leGeneres.inverse_transform(logReg.predict(pred))